In [2]:
import os
%pwd

'/media/sasan/Common/ML/EtoE/research'

In [3]:
os.chdir('../')
%pwd

'/media/sasan/Common/ML/EtoE'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    evaluation_metrics_file: Path
    target_column: str

In [12]:
from RedWineQualityPrediction.constanst import *
from RedWineQualityPrediction.utils.common import read_yaml, create_derectories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_derectories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_derectories([config.root_dir])
        
        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            evaluation_metrics_file=config.evaluation_metrics_file,
            target_column=schema.name
        )

In [11]:
import pandas as pd
import os
from RedWineQualityPrediction import logger
from sklearn.linear_model import ElasticNet
import joblib
from urllib.parse import urlparse
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [13]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, predict):
        rmse = np.sqrt(mean_squared_error(actual, predict))
        mae = mean_absolute_error(actual, predict)
        r2 = r2_score(actual, predict)
        return rmse, mae, r2
    
    
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)
        
        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
        
        # save metrics
        scores = {"RMSE": rmse, "MAE": mae, "R2": r2}
        save_json(path=Path(self.config.evaluation_metrics_file), data=scores)
        

In [14]:
try:
    cofig = ConfigurationManager()
    model_eval_mgr = cofig.get_model_evaluation_config()
    model_eval = ModelEvaluation(model_eval_mgr)
    model_eval.save_results()
except Exception as error:
    raise error

[2025-02-12 16:27:29,276: INFO: common: Reading the yaml file from ::     config/config.yml]
[2025-02-12 16:27:29,279: INFO: common: Reading the yaml file from ::     params.yml]
[2025-02-12 16:27:29,285: INFO: common: Reading the yaml file from ::     schema.yml]
[2025-02-12 16:27:29,287: INFO: common: Directory created at :: artifacts]
[2025-02-12 16:27:29,289: INFO: common: Directory created at :: artifacts/model_evaluation]
[2025-02-12 16:27:29,419: INFO: common: Data saved in json format at :: artifacts/model_evaluation/evaluation_metrics.json]
